In [ ]:
%load_ext autoreload
%autoreload 2

# Exercise 6

<img src='./images/06.png' width=800>

In [ ]:
import os
import mlflow
os.environ['MLFLOW_TRACKING_URI'] = './mlruns03_6'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise_6')

2025/04/18 15:07:16 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_5' does not exist. Creating a new experiment.


<Experiment: artifact_location='/kaggle/working/mlruns/360360039686392042', creation_time=1744988836419, experiment_id='360360039686392042', last_update_time=1744988836419, lifecycle_stage='active', name='Exercise_5', tags={}>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.optim as optim
from torchinfo import summary
from utils import train_network, accuracy_score_wrapper, weight_reset


In [ ]:
batch_size = 256

In [ ]:

C, H, W = 3, 32, 32
num_pixels = H * W

permutation = torch.randperm(num_pixels)

def fixed_shuffle(img):
    """
    Expects a tensor image of shape [C, H, W]
    """
    # Flatten the spatial dimensions: for each channel, create a (H*W) vector.
    # img.view(C, -1) will give a [C, H*W] tensor.
    img_reshaped = img.view(C, -1)
    
    # Apply the fixed permutation to each channel
    img_shuffled = img_reshaped[:, permutation]
    
    # Reshape back to [C, H, W]
    return img_shuffled.view(C, H, W)

# Now create a transform pipeline using Lambda
shuffle_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(fixed_shuffle),
])


In [ ]:
cifar_train_shuffletransform = torchvision.datasets.CIFAR10(
    './data_cifar_shuffletransform',
    download=True,
    transform=shuffle_transform,
    # target_transform=lambda x: torch.tensor(x),
    train=True,)
cifar_test_shuffletransform = torchvision.datasets.CIFAR10(
    './data_cifar_shuffletransform',
    download=True,
    transform=shuffle_transform,
    # target_transform=lambda x: torch.tensor(x),
    train=False,)

cifar_train_loader_shuffle = DataLoader(
    cifar_train_shuffletransform,
    shuffle=True,
    batch_size=batch_size,
    )

cifar_test_loader_shuffle = DataLoader(
    cifar_test_shuffletransform,
    batch_size=batch_size,
    )

100%|██████████| 170M/170M [00:04<00:00, 34.5MB/s] 


Extracting ./data_cifar_shuffletransform/cifar-10-python.tar.gz to ./data_cifar_shuffletransform
Files already downloaded and verified


In [ ]:
cifar_train = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transforms.ToTensor(),
    train=True,)
cifar_test = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transforms.ToTensor(),
    train=False,)

cifar_train_loader = DataLoader(
    cifar_train,
    shuffle=True,
    batch_size=batch_size,
    )

cifar_test_loader = DataLoader(
    cifar_test,
    batch_size=batch_size,
    )

100%|██████████| 170M/170M [00:04<00:00, 35.3MB/s] 


Extracting ./data_cifar/cifar-10-python.tar.gz to ./data_cifar
Files already downloaded and verified


In [ ]:
train_loaders = (cifar_train_loader, cifar_train_loader_shuffle)
test_loaders = (cifar_test_loader, cifar_test_loader_shuffle)
name_experiment = (
    'build_model_cifar',
    'build_model_cifar_shuffletransform',
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs = 20
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
}

In [ ]:
# C = cifar_train[0][0].shape[0]
C = 3
filter = 16
K = 3
# w = cifar_train[0][0].shape[1]
# h = cifar_train[0][0].shape[2]
w , h = 32, 32
# classes = cifar_train[0][1].shape[0]
classes = 10
def build_model(num_conv_layers,
                num_pool_layers,
                num_hidden_layer=2, 
                init_hidden_size=512, 
                decay_factor=2,
                activation=nn.ReLU(),
                out_channels=32):
    layers =[]
    in_channels = C
    # out_channels = 32
    if num_pool_layers:
        pool_interval = max(1, num_conv_layers // (num_pool_layers + 1))
    else:
        pool_interval = num_conv_layers + 1
    
    currnet_pool_rounds = 0
    for i in range(num_conv_layers):
        layers.append(nn.Conv2d(
            in_channels=in_channels, 
            out_channels=out_channels,
            kernel_size=3,
            padding=3//2))
        layers.append(activation)
        in_channels = out_channels
        if (i+1) % pool_interval == 0 and currnet_pool_rounds < num_pool_layers:
            layers.append(nn.MaxPool2d(kernel_size=2))
            currnet_pool_rounds += 1
            out_channels *= 2
    final_w = w // (2 ** num_pool_layers)
    final_h = h // (2 ** num_pool_layers)
    fc_layers = []
    # Compute the number of features after flattening.
    in_features = in_channels * final_w * final_h

    fc_layers.append(nn.Flatten())

    if num_hidden_layer == 0:
        # Directly classify without extra hidden layers.
        fc_layers.append(nn.Linear(in_features, classes))
    else:
        # First FC layer: from flattened output to initial hidden size.
        fc_layers.append(nn.Linear(in_features, init_hidden_size))
        fc_layers.append(nn.ReLU(inplace=True))
        
        # Set the current hidden size that will be reduced in subsequent layers.
        current_hidden_size = init_hidden_size

        # Add additional hidden layers with decreasing size.
        for layer in range(1, num_hidden_layer):
            # Compute new hidden size with decay.
            new_hidden_size = max(10, current_hidden_size // decay_factor)
            fc_layers.append(nn.Linear(current_hidden_size, new_hidden_size))
            fc_layers.append(nn.ReLU(inplace=True))
            current_hidden_size = new_hidden_size

        # Final classification layer from the last hidden dimension to the number of classes.
        fc_layers.append(nn.Linear(current_hidden_size, classes))

    classifier = nn.Sequential(*fc_layers)
    model = nn.Sequential(*layers, classifier)
    return model

In [ ]:
model = build_model(num_conv_layers=4,
        num_pool_layers=2,
        num_hidden_layer=2, 
        init_hidden_size=512, 
        decay_factor=2,
        activation=nn.ReLU(),
        out_channels=256)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
params['optimizer'] = optimizer.defaults
with open('model_summary.txt', 'w') as f:
    f.write(str(summary(model)))
for i in range(2):
    print(i)
    train_loader = train_loaders[i]
    test_loader = test_loaders[i]    
    with mlflow.start_run(nested=True, run_name=name_experiment[i]):
        mlflow.log_params(params)
        mlflow.log_artifact('model_summary.txt')
        model.apply(weight_reset)
        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=test_loader,
            epochs=epochs,
            device=device,
            score_funcs=score_funcs,
            )

<img src="./images/valid_acc_shuffled.png">

<img src="./images/valid_loss_shufled.png">